Working file for https://redmine.deriv.cloud/issues/95135#Enable_vanilla_synthetics_on_real<br>
Tested on 10/07/2023 on QA142 using ETH as account currency

In [34]:
import requests
import websocket
import json
from websocket import create_connection
import time
import pandas as pd
import numpy as np
from scipy.stats import norm
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option('display.max_colwidth',300)
from IPython.display import clear_output
import random
from scipy.optimize import fsolve

# from VanillasFunctions import add,barrier_check

In [35]:
# Volatility index parameters such as vol, displayed decimals
assets=pd.read_csv("assets.csv")
#API URL
master_URL = "wss://qa142.deriv.dev/websockets/v3?app_id=1007&l=EN&brand=deriv"
#api token
api_token = "9k5qPPtGzwuPPAr"

In [36]:
def barrier_check (symbol, duration, duration_unit, contract_type): 
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            if duration_unit == "d":
                barrier = "10000"
            else:
                barrier = "+0"
            msg = {
                "proposal": 1,
                "amount" : 0.05,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "ETH",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            print('sendbar')
            ws.send(json.dumps(msg))
        else:
            print('here')
            amount = 0.05
            if 'error' in message.keys():
                barrier = message['error']['details']['barrier_choices'][int(len(message['error']['details']['barrier_choices'])/2)]
                if message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    print('amount =',amount)
                    print(barrier)
                    print(contract_type)
                    print(duration)
                    print(duration_unit)
                    print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "ETH",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print('sent')
                ws.send(json.dumps(msg))   
            else:
                return_json.append(message)
                ws.close()
    def on_close(ws, close_status_code, close_msg):
        print()             
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    barriers = return_json[-1]['proposal']['barrier_choices']
    return return_json[-1]

In [37]:
def calc_barriers (pd_series): #(spot, symbol, duration, duration_unit):
    
    spot = float(pd_series.Spot)
    
    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    digits = int( assets[assets['Asset']== symbol].digits )
    
    sigma = float( assets[assets['Asset']==symbol].IV )  
    
    n_digits = round( - np.log10(spot * sigma * np.sqrt(60/365/86400)) , 0) - 1
    
    spot = round(spot/10**-n_digits)*10**-n_digits
    
    t = (int(duration[1]) - int(duration[0])) / 365 / 86400       
    
    # Uncomment this section to match all API barriers to self calculated barriers
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if duration_unit != 'd':
        
        deltas = np.arange(1,10,2)/10 
    
        real_strikes = spot * np.exp( (sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(deltas)) )
    
        displayed_barriers = np.where( deltas > 0.5, np.round((real_strikes - spot)*10**digits)/10**digits , np.round((real_strikes - spot)*10**digits)/10**digits )
        
    else:
        
        temp_barriers = []
        
        displayed_barriers = []

        central_strike = round( spot / (10 ** -n_digits) ) * (10 ** - n_digits )

        max_strike =  np.floor( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.1)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        min_strike =  np.ceil( (spot * np.exp((sigma**2)/2*t - (sigma * np.sqrt(t)  * norm.ppf(0.9)))) / (10 ** -n_digits ) ) * (10 ** -n_digits )

        strike_step_1 = round( (central_strike - min_strike) / 5 / 10 ) * 10

        strike_step_2 = round( (max_strike - central_strike) / 5 / 10 ) * 10 

        for i in range(5):
            temp_barriers.append(min_strike + strike_step_1*i)

        for i in range(5):
            temp_barriers.append(central_strike + strike_step_2*i)

        temp_barriers.append(max_strike)

        for strike in temp_barriers:
            if strike not in displayed_barriers:
                displayed_barriers.append(strike)


    return displayed_barriers
        

In [38]:
def calc_ppp_ask (pd_series, t_step, s_step):

    symbol = pd_series.Asset
    
    duration = pd_series.Start_expiry
    
    duration_unit = pd_series.Duration[-1]
    
    stake = pd_series.Stake

    vol_charge = 1.025
 
    t = ( int(duration[1]) - int(duration[0]) + t_step ) / 365 / 86400  
    
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        print('rounded')
        t -= 2 / 365 / 86400

    r = pd_series.Discount_rate 
    
    contract_type = pd_series.Contract_type

    number_of_contracts = pd_series.API_PPP
        
    if s_step == 0:
        
        spot = float(pd_series.Spot)

        strike = pd_series.Barrier
    
    else:
        
        spot = float(tick_history(int(duration[0]), symbol, s_step))
        
        if duration_unit != 'd':
            
            strike = spot + float(pd_series.Displayed_barrier)
            
        else:
            
            strike = pd_series.Barrier
        
    sigma = float( assets[assets['Asset']==symbol].IV )
    
    if duration_unit != 'd':
        
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
        
    else:
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2
        
    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    
    bs_price = (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2)))
    
    print(symbol,'s', spot, 'k',strike, r, t, vol,  phi,  'p',number_of_contracts, stake, 'd', (norm.cdf(phi*d1)))
    
    return stake / ( bs_price + delta_charge + bs_markup )

In [39]:
def tick_history (tick,symbol,t):
    
    if 'R' in symbol:
        r = 2
    else:
        r = 1
        
    temp_json = []
    
    def on_open(ws):
        omsg = {
          "ticks_history": symbol,
          "adjust_start_time": 1,
          "count": 5,
          "end": tick+t*r,
          "start": 1,
          "style": "ticks"
        }
        ws.send(json.dumps(omsg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(message)
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]['history']['prices'][-1]
    

In [40]:
def get_proposal (contract_id): 
    
    contract_id = int(contract_id)
    
    return_json = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        
        ws.send(json.dumps(msg))


    def on_message(ws, message):

        message = json.loads(message)
        print(message)
        if 'authorize' in message:
            
            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_id
            }
            
            ws.send(json.dumps(msg))
            
        else:
                    
            return_json.append(message)
            
            ws.close()

    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1]

In [41]:
def get_barriers (symbol, duration, duration_unit, contract_type): 
    barrier = 0
    if duration_unit == "d":
        barrier = "10000"
    else:
        barrier = "+0"
    return_json = []
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
    def on_message(ws, message):
        message = json.loads(message)
        if duration_unit == "d":
            barrier = "10000"
        else:
            barrier = "+0" 
        print('\n',message)
        if 'authorize' in message:
            msg = {
                "proposal": 1,
                "amount" :0.05,
                "barrier": barrier,
                "basis": "stake",
                "contract_type": contract_type,
                "currency": "ETH",
                "duration": duration,
                "duration_unit": duration_unit,
                "symbol": symbol
            }
            ws.send(json.dumps(msg))
        else:
            if 'error' in message.keys():
                amount = float(message['error']['details']['min_stake'])
                if message['error']['details']['field'] == 'barrier':
                    barrier = message['error']['details']['barrier_choices'][np.random.randint(len(message['error']['details']['barrier_choices']))]
                    amount = float(message['error']['details']['max_stake'])
                elif message['error']['details']['field'] == 'amount':
                    amount = float(message['error']['details']['max_stake'])
                    barrier = message['echo_req']['barrier']

                print(amount)
                print(barrier)
                print(contract_type)
                print(duration)
                print(duration_unit)
                print(symbol)
                msg = {
                    "proposal": 1,
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "ETH",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                }
                print(msg)
                ws.send(json.dumps(msg))   

            else:

                return_json.append(message)
                ws.close()
            
    def on_close(ws, close_status_code, close_msg):
        print()             
 
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    barriers = return_json[-1]['proposal']['barrier_choices']
    
    if 'qa' in master_URL:
        
        return barriers, 0#return_json[-1]['proposal']['pricing_args']['discount_rate']
    
    else:
        
        return barriers, 'prod_api'

In [42]:
def get_contract (symbol, duration, duration_unit, contract_type): 
#     temp_unit = [duration_unit]
    return_json = []

    return_barrier = []
    
    return_discount_rate = []

    contract_ID = []
    
    def on_open(ws):
        
        msg = {
            "authorize": api_token,
        }
        ws.send(json.dumps(msg))

    def on_message(ws, message):
        print('\n',message)
        barrier = 0
        
        amount = 0.05
        
        message = json.loads(message)
        
#         print('\n',message)
#         print('here1')
        if 'authorize' in message:
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)
            barrier = barriers[0][np.random.randint(len(barriers[0]))]
            return_barrier.append(barrier)
            return_discount_rate.append(barriers[1])
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "ETH",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            ws.send(json.dumps(msg))    

        elif 'error' in message.keys():
            if 'code' in message['error'].keys():
                print('here')
                print(message['error']['message'])
                print(message['error']['message'].split()[-1][:-1])
                print(amount)
                amount = float(message['error']['message'].split()[-1][:-1]) 
    
            print('here2',amount)    
            barriers = get_barriers(symbol, duration, duration_unit, contract_type)

            barrier = barriers[0][np.random.randint(len(barriers[0]))]

            return_barrier.append(barrier)

            return_discount_rate.append(barriers[1])
            
            print (symbol)
            
            msg = {
                "buy": "1",
                "parameters" :
                {
                    "amount": amount,
                    "barrier": barrier,
                    "basis": "stake",
                    "contract_type": contract_type,
                    "currency": "ETH",
                    "duration": duration,
                    "duration_unit": duration_unit,
                    "symbol": symbol
                },
                "price": amount
            }
            print('msgcre')
            print(msg)
            ws.send(json.dumps(msg))
            print('sent')
        elif 'buy' in message:
            print('bought')
            contract_ID.append(message['buy']['contract_id'])

            msg = {
              "proposal_open_contract": 1,
              "contract_id": contract_ID[-1]
            }

            ws.send(json.dumps(msg))
            
        else:
            return_json.append(message)
            
            ws.close()
                                          
    def on_close(ws, close_status_code, close_msg):
        print()

    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return return_json[-1], return_discount_rate[-1], return_barrier[-1]

In [55]:
def calc_ppp_bid (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['bid_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['current_spot_time'] + t_step ) / 365 / 86400  
        
    if t < 5 * 60 / 365 / 86400 and 'R' in symbol:
        t -= 2 / 365 / 86400
        
    if t > 1 / 365:
        
        r =0#proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    
    
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['current_spot'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(int(proposal['proposal_open_contract']['current_spot_time']), symbol, s_step))
        
        if t > 1 / 365:
        
            strike = float(proposal['proposal_open_contract']['barrier'])

        else:

            strike = spot + float(proposal['proposal_open_contract']['barrier']) - float(proposal['proposal_open_contract']['current_spot'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

        
    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
    print(spot,strike,r, t, phi,  number_of_contracts, stake)

    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))

    return ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup) * number_of_contracts
    

In [44]:
def sell_contract(contract_id):
    
    contract_id = int(contract_id)
    
    temp_json = []
    
    def on_open(ws):
        msg = {
            "authorize": api_token,
            "req_id": 1
        }
        ws.send(json.dumps(msg))
        
    def on_message(ws, message):

        message = json.loads(message)
        print('\n',message)
        if 'authorize' in message:
            
            msg = {
              "sell": contract_id,
              "price": 0
            }
            ws.send(json.dumps(msg))
            
        else:        
            temp_json.append(float(message['sell']['sold_for']))
            ws.close()
    
    def on_close(ws, close_status_code, close_msg):
        print()
        
    ws = websocket.WebSocketApp(master_URL, on_message = on_message, on_open = on_open, on_close=on_close)
    ws.run_forever()
    
    return temp_json[-1]

In [56]:
def calc_ppp_sold (proposal, t_step, s_step):

    symbol = proposal['proposal_open_contract']['underlying']
    
    stake = float(proposal['proposal_open_contract']['sell_price'])

    vol_charge = 0.975
 
    t = ( proposal['proposal_open_contract']['date_expiry'] - proposal['proposal_open_contract']['exit_tick_time'] + t_step) / 365 / 86400  

    if t > 1 / 365:
        
        r =0# proposal['proposal_open_contract']['pricing_args']['discount_rate']
        
        spot_spread = float( assets[assets['Asset']==symbol].SSD )
    
        bs_markup = float( assets[assets['Asset']==symbol].BSD )
        
    else:
        
        r = 0
                
        spot_spread = float( assets[assets['Asset']==symbol].SS )
    
        bs_markup = float( assets[assets['Asset']==symbol].BS )
    
    contract_type = proposal['proposal_open_contract']['contract_type']

    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
        
    if s_step == 0:
        
        spot = float(proposal['proposal_open_contract']['exit_tick'])

        strike = float(proposal['proposal_open_contract']['barrier'])
    
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    sigma = float( assets[assets['Asset']==symbol].IV )

    if contract_type == "VANILLALONGCALL":
        
        phi = 1
        
        delta_charge = (norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t)))) * spot_spread / 2

    elif  contract_type == "VANILLALONGPUT":
        
        phi = -1
        
        delta_charge = (abs(norm.cdf((np.log(spot/strike) + (sigma**2/2)*t)/(sigma*np.sqrt(t))) - 1)) * spot_spread / 2
    
        
    print('s',spot,'k',strike,'r',r,'t', t, phi,'n',  number_of_contracts,'s', stake)


    vol = sigma * vol_charge

    d1 = (np.log(spot/strike) + (vol**2/2)*t)/(vol*np.sqrt(t))

    d2 = (d1 - vol*np.sqrt(t))
    
    price = ( (phi*np.exp(-r*t)*(spot*norm.cdf(phi*d1) - strike*norm.cdf(phi*d2))) - delta_charge - bs_markup)
    print(price)
    return max(price * number_of_contracts,0)
    


In [46]:
def calc_ppp_exp (proposal, t_step, s_step):
    stake = float(proposal['proposal_open_contract']['sell_price'])
    contract_type = proposal['proposal_open_contract']['contract_type']
    number_of_contracts = float(proposal['proposal_open_contract']['number_of_contracts'])
    if s_step == 0:
        spot = float(proposal['proposal_open_contract']['exit_tick'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    else:
        spot = float(tick_history(proposal['proposal_open_contract']['exit_tick_time'], symbol, s_step))
        
#         strike = spot - float(proposal['proposal_open_contract']['exit_tick']) + float(proposal['proposal_open_contract']['barrier'])
        strike = float(proposal['proposal_open_contract']['barrier'])
    
    if contract_type == "VANILLALONGCALL":
        phi = 1
    elif  contract_type == "VANILLALONGPUT":
        phi = -1
    print(spot,strike, phi,  number_of_contracts, stake)
    return max( (spot - strike) * phi * number_of_contracts, 0)

In [30]:
# barrier_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Duration', 'Start_expiry', 'API_barrier_choice', 'Calc_barrier_choices', 'json'])
contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]
time_units = ('m', 'h', 'd')
while True:
    clear_output(wait = True)
    print(len(barrier_df))
    if len(barrier_df) > 15 - 1:
        break
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        
        max_duration = 1440

    elif duration_unit == 'h':
        
        max_duration = 24

    elif duration_unit == 'd':
        
        max_duration = 365

    i = np.random.randint(5)
    
    if i < 2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1,max_duration)
        
    price_proposal = barrier_check(asset, duration, duration_unit, contract_type)
    
    barrier_df.loc[len(barrier_df)] = [ price_proposal['echo_req']['symbol'], price_proposal['echo_req']['contract_type'], price_proposal['proposal']['spot'], \
                                        price_proposal['proposal']['spot_time'], str(price_proposal['echo_req']['duration']) + price_proposal['echo_req']['duration_unit'], \
                                        [ price_proposal['proposal']['date_start'], price_proposal['proposal']['date_expiry'] ], \
                                        [float(x) for x in price_proposal['proposal']['barrier_choices']], 0, price_proposal ]
    
clear_output(wait = True)
barrier_df

,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json
0,R_25,VANILLALONGCALL,9989.8450,1689910000,5h,"[1689910000, 1689928000]","[-76.07, -31.09, 0.18, 31.55, 77.02]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+0.180', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'ETH', 'duration': 5, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_25'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices': [..."
1,R_100,VANILLALONGCALL,10082.6900,1689910002,1124m,"[1689910002, 1689977442]","[-571.0, -231.4, 10.8, 259.0, 628.1]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+10.80', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'ETH', 'duration': 1124, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_100'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices..."
2,R_75,VANILLALONGPUT,10024.0546,1689910004,1h,"[1689910004, 1689913604]","[-101.851, -41.614, 0.321, 42.432, 103.548]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+0.3210', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'ETH', 'duration': 1, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_75'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices': [..."
3,1HZ10V,VANILLALONGCALL,10039.5300,1689910006,68d,"[1689910006, 1695859199]","[9510.0, 9620.0, 9730.0, 9840.0, 9950.0, 10040.0, 10160.0, 10280.0, 10400.0, 10520.0, 10620.0]",0,"{'echo_req': {'amount': 0.05, 'barrier': '10040.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'ETH', 'duration': 68, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ10V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choice..."
4,1HZ50V,VANILLALONGPUT,9746.6700,1689910008,1h,"[1689910008, 1689913608]","[-66.0, -27.0, 0.1, 27.4, 66.8]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+0.10', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'ETH', 'duration': 1, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ50V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices': [..."
5,1HZ50V,VANILLALONGPUT,9747.0500,1689910009,365d,"[1689910010, 1721519999]","[5800.0, 6580.0, 7360.0, 8140.0, 8920.0, 9700.0, 11920.0, 14140.0, 16360.0, 18580.0, 20800.0]",0,"{'echo_req': {'amount': 0.05, 'barrier': '9700.00', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'ETH', 'duration': 365, 'duration_unit': 'd', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ50V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices..."
6,R_10,VANILLALONGPUT,10030.6450,1689910010,1h,"[1689910012, 1689913612]","[-13.72, -5.61, 0.01, 5.63, 13.75]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+0.010', 'basis': 'stake', 'contract_type': 'VANILLALONGPUT', 'currency': 'ETH', 'duration': 1, 'duration_unit': 'h', 'product_type': 'basic', 'proposal': 1, 'symbol': 'R_10'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices': ['..."
7,1HZ75V,VANILLALONGCALL,10187.4500,1689910013,270m,"[1689910013, 1689926213]","[-218.4, -89.1, 1.5, 92.8, 226.1]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+1.50', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'ETH', 'duration': 270, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ75V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices'..."
8,1HZ100V,VANILLALONGCALL,9474.1800,1689910015,1m,"[1689910015, 1689910075]","[-16.8, -6.9, 0.0, 6.9, 16.8]",0,"{'echo_req': {'amount': 0.05, 'barrier': '+0.00', 'basis': 'stake', 'contract_type': 'VANILLALONGCALL', 'currency': 'ETH', 'duration': 1, 'duration_unit': 'm', 'product_type': 'basic', 'proposal': 1, 'symbol': '1HZ100V'}, 'msg_type': 'proposal', 'proposal': {'ask_price': 0.05, 'barrier_choices':..."
9,R_25,VANILLALONG

In [31]:
Calc_barrier_choices = [] 

for i in barrier_df.index:
    
    Calc_barrier_choices.append( calc_barriers(barrier_df.iloc[i]) )

barrier_df.Calc_barrier_choices = Calc_barrier_choices

print(len(barrier_df), 'Test cases','\nMismatches =',\
      len(barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]),\
      '\nMismatches listed below :')

barrier_df[barrier_df.API_barrier_choice.map(np.abs).map(sum) - barrier_df.Calc_barrier_choices.map(np.abs).map(sum) > 0.00000001]


15 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Duration,Start_expiry,API_barrier_choice,Calc_barrier_choices,json


In [49]:
# PPP_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Calc_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    clear_output(wait=True)
    print(len(PPP_df))
    
    if len(PPP_df)>20-1:
        break
        
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
    duration_unit = random.choice(time_units)
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
    
    if i<2:
        
        duration = max_duration**i
        
    else:
        
        duration = np.random.randint(1, max_duration)
#     duration = np.random.randint(1, 5)
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    PPP_df.loc[len(PPP_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, proposal]
                               

#     clear_output(wait=True)
clear_output(wait=True)

PPP_df


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json
0,1HZ75V,VANILLALONGCALL,10199.6900,1689910223,0.050000,1h,+105.60,10305.290,"[1689910223, 1689913823]",319,0,0.011837,0,"{'echo_req': {'contract_id': 319, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10305.29', 'barrier_count': 1, 'bid_price': 0.04151072, 'buy_price': 0.05, 'contract_id': 319, 'contract_type': 'VANILLALONGCALL', 'c..."
1,1HZ10V,VANILLALONGPUT,10040.2100,1689910226,0.050000,6h,+0.00,10040.210,"[1689910227, 1689931827]",339,0,0.004653,0,"{'echo_req': {'contract_id': 339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10040.21', 'barrier_count': 1, 'bid_price': 0.04705138, 'buy_price': 0.05, 'contract_id': 339, 'contract_type': 'VANILLALONGPUT', 'cu..."
2,1HZ100V,VANILLALONGCALL,9429.4900,1689910230,0.050000,1440m,+274.80,9704.290,"[1689910230, 1689996630]",359,0,0.000521,0,"{'echo_req': {'contract_id': 359, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9704.29', 'barrier_count': 1, 'bid_price': 0.04558062, 'buy_price': 0.05, 'contract_id': 359, 'contract_type': 'VANILLALONGCALL', 'cu..."
3,R_100,VANILLALONGCALL,10097.0900,1689910234,0.050000,16h,+238.30,10335.390,"[1689910234, 1689967834]",379,0,0.000595,0,"{'echo_req': {'contract_id': 379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10335.39', 'barrier_count': 1, 'bid_price': 0.04553957, 'buy_price': 0.05, 'contract_id': 379, 'contract_type': 'VANILLALONGCALL', 'c..."
4,1HZ25V,VANILLALONGCALL,10165.1900,1689910241,0.009686,1d,10120.00,10120.000,"[1689910241, 1690070399]",399,0,0.000098,0,"{'echo_req': {'contract_id': 399, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10120.00', 'barrier_count': 1, 'bid_price': 0.00934307, 'buy_price': 0.009686, 'contract_id': 399, 'contract_type': 'VANILLALONGCALL'..."
5,R_75,VANILLALONGPUT,9987.3180,1689910244,0.050000,19h,-431.9270,9555.391,"[1689910245, 1689978645]",419,0,0.002739,0,"{'echo_req': {'contract_id': 419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9555.3910', 'barrier_count': 1, 'bid_price': 0.04163655, 'buy_price': 0.05, 'contract_id': 419, 'contract_type': 'VANILLALONGPUT', 'c..."
6,1HZ25V,VANILLALONGCALL,10165.2300,1689910249,0.050000,81d,9160.00,9160.000,"[1689910249, 1696982399]",459,0,0.000044,0,"{'echo_req': {'contract_id': 459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9160.00', 'barrier_count': 1, 'bid_price': 0.04931765, 'buy_price': 0.05, 'contract_id': 459, 'contract_type': 'VANILLALONGCALL', 'cu..."
7,R_10,VANILLALONGPUT,10031.3320,1689910252,0.050000,655m,-45.210,9986.122,"[1689910253, 1689949553]",499,0,0.027199,0,"{'echo_req': {'contract_id': 499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9986.122', 'barrier_count': 1, 'bid_price': 0.04154788, 'buy_price': 0.05, 'contract_id': 499, 'contract_type': 'VANILLALONGPUT', 'cu..."
8,R_50,VANILLALONGPUT,10127.8675,1689910256,0.050000,7d,9940.0000,9940.000,"[1689910257, 1690588799]",539,0,0.000231,0,"{'echo_req': {'contract_id': 539, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9940.0000', 'barrier_count': 1, 'bid_price': 0.04671643, 'buy_price': 0.05, 'contract_id': 539, 'contract_type': 'VANILLALONGPUT', 'c..."
9,R_100,VANILLALONGCALL,10105.0700,1689910260,0.050000,251d,5000.00,50

In [50]:
Calc_ppp = [] 

for i in PPP_df.index:
    
    Calc_ppp.append( calc_ppp_ask(PPP_df.iloc[i], 0, 0) )

PPP_df.Calc_PPP = Calc_ppp

clear_output(wait = True)

print(len(PPP_df), 'Test cases','\nMismatches =',\
      len(PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]),\
      '\nMismatches listed below :')

df = PPP_df[(PPP_df.API_PPP) - np.round(PPP_df.Calc_PPP,10) != 0]

df

20 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Calc_PPP,json


#### Mismatches below are due assumming discount rate = 0 in calculations here

In [53]:
# Bid_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'Bid_time', 'API_bid', 'Calc_bid','bid_prop','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m', 'h', 'd')

while True:
    
    print(len(Bid_df))
    
    if len(Bid_df)>20-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
#     asset = 'R_10'
    duration_unit = random.choice(time_units)

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Bid_df.loc[len(Bid_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, 0, 0, proposal]
                               

    clear_output(wait=True)
Bid_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ25V,VANILLALONGPUT,10177.6700,1689910968,0.05,14h,-129.30,10048.3700,"[1689910969, 1689961369]",1179,0,0.009503,0,0,0,0,"{'echo_req': {'contract_id': 1179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10048.37', 'barrier_count': 1, 'bid_price': 0.04217838, 'buy_price': 0.05, 'contract_id': 1179, 'contract_type': 'VANILLALONGPUT', '..."
1,1HZ50V,VANILLALONGPUT,9714.3300,1689910972,0.05,15h,+2.10,9716.4300,"[1689910973, 1689964973]",1199,0,0.000601,0,0,0,0,"{'echo_req': {'contract_id': 1199, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9716.43', 'barrier_count': 1, 'bid_price': 0.04740603, 'buy_price': 0.05, 'contract_id': 1199, 'contract_type': 'VANILLALONGPUT', 'c..."
2,1HZ10V,VANILLALONGCALL,10040.1700,1689910977,0.05,831m,+0.10,10040.2700,"[1689910977, 1689960837]",1239,0,0.003072,0,0,0,0,"{'echo_req': {'contract_id': 1239, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10040.27', 'barrier_count': 1, 'bid_price': 0.0475535, 'buy_price': 0.05, 'contract_id': 1239, 'contract_type': 'VANILLALONGCALL', '..."
3,1HZ100V,VANILLALONGPUT,9362.3400,1689910981,0.05,1391m,-238.10,9124.2400,"[1689910981, 1689994441]",1259,0,0.000510,0,0,0,0,"{'echo_req': {'contract_id': 1259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9124.24', 'barrier_count': 1, 'bid_price': 0.04573801, 'buy_price': 0.05, 'contract_id': 1259, 'contract_type': 'VANILLALONGPUT', 'c..."
4,1HZ50V,VANILLALONGPUT,9718.3300,1689910985,0.05,20h,-118.00,9600.3300,"[1689910985, 1689982985]",1299,0,0.001067,0,0,0,0,"{'echo_req': {'contract_id': 1299, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9600.33', 'barrier_count': 1, 'bid_price': 0.0456916, 'buy_price': 0.05, 'contract_id': 1299, 'contract_type': 'VANILLALONGPUT', 'cu..."
5,R_10,VANILLALONGPUT,10033.4150,1689910988,0.05,301d,11070.000,11070.0000,"[1689910989, 1715990399]",1339,0,0.000045,0,0,0,0,"{'echo_req': {'contract_id': 1339, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '11070.000', 'barrier_count': 1, 'bid_price': 0.04952067, 'buy_price': 0.05, 'contract_id': 1339, 'contract_type': 'VANILLALONGPUT', ..."
6,1HZ50V,VANILLALONGPUT,9717.0200,1689910993,0.05,1265m,-297.40,9419.6200,"[1689910993, 1689986893]",1379,0,0.003994,0,0,0,0,"{'echo_req': {'contract_id': 1379, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9419.62', 'barrier_count': 1, 'bid_price': 0.04162685, 'buy_price': 0.05, 'contract_id': 1379, 'contract_type': 'VANILLALONGPUT', 'c..."
7,1HZ100V,VANILLALONGPUT,9357.6900,1689910997,0.05,1153m,-218.00,9139.6900,"[1689910997, 1689980177]",1419,0,0.000562,0,0,0,0,"{'echo_req': {'contract_id': 1419, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9139.69', 'barrier_count': 1, 'bid_price': 0.04572696, 'buy_price': 0.05, 'contract_id': 1419, 'contract_type': 'VANILLALONGPUT', 'c..."
8,R_100,VANILLALONGCALL,10132.5900,1689911000,0.05,14h,+8.10,10140.6900,"[1689911001, 1689961401]",1459,0,0.000309,0,0,0,0,"{'echo_req': {'contract_id': 1459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10140.69', 'barrier_count': 1, 'bid_price': 0.04750117, 'buy_price': 0.05, 'contract_id': 1459, 'contract_type': 'VANILLALONGCALL', ..."
9,R_75,

In [59]:
calc_bid = [] 
bid_time = []
api_bid = []
bid_prop = []
for i in Bid_df.index:   
    if i >= len(bid_time):       
        clear_output(wait = True)
        proposal = get_proposal(Bid_df.loc[i].ID)
        bid_time.append(proposal['proposal_open_contract']['current_spot_time'])
        api_bid.append(float(proposal['proposal_open_contract']['bid_price']))
        calc_bid.append( calc_ppp_bid(proposal, 0, 0) )       
        bid_prop.append(proposal)   
Bid_df.Bid_time = bid_time
Bid_df.API_bid = api_bid
Bid_df.Calc_bid = calc_bid
Bid_df.bid_prop = bid_prop
clear_output(wait = True)
print(len(Bid_df), 'Test cases','\nMismatches =',\
      len(Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,8) != 0]),\
      '\nMismatches listed below :')
df = Bid_df[Bid_df.API_bid - np.round(Bid_df.Calc_bid,8) != 0]
df

20 Test cases 
Mismatches = 9 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,Bid_time,API_bid,Calc_bid,bid_prop,json
0,1HZ25V,VANILLALONGPUT,10177.6700,1689910968,0.05,14h,-129.30,10048.3700,"[1689910969, 1689961369]",1179,0,0.009503,1689916362,0.075174,0.075177,"{'echo_req': {'contract_id': 1179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10048.37', 'barrier_count': 1, 'bid_price': 0.07517434, 'buy_price': 0.05, 'contract_id': 1179, 'contract_type': 'VANILLALONGPUT', '...","{'echo_req': {'contract_id': 1179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10048.37', 'barrier_count': 1, 'bid_price': 0.04217838, 'buy_price': 0.05, 'contract_id': 1179, 'contract_type': 'VANILLALONGPUT', '..."
3,1HZ100V,VANILLALONGPUT,9362.3400,1689910981,0.05,1391m,-238.10,9124.2400,"[1689910981, 1689994441]",1259,0,0.000510,1689916367,0.016308,0.016308,"{'echo_req': {'contract_id': 1259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9124.24', 'barrier_count': 1, 'bid_price': 0.01630762, 'buy_price': 0.05, 'contract_id': 1259, 'contract_type': 'VANILLALONGPUT', 'c...","{'echo_req': {'contract_id': 1259, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9124.24', 'barrier_count': 1, 'bid_price': 0.04573801, 'buy_price': 0.05, 'contract_id': 1259, 'contract_type': 'VANILLALONGPUT', 'c..."
8,R_100,VANILLALONGCALL,10132.5900,1689911000,0.05,14h,+8.10,10140.6900,"[1689911001, 1689961401]",1459,0,0.000309,1689916374,0.045041,0.045042,"{'echo_req': {'contract_id': 1459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10140.69', 'barrier_count': 1, 'bid_price': 0.04504125, 'buy_price': 0.05, 'contract_id': 1459, 'contract_type': 'VANILLALONGCALL', ...","{'echo_req': {'contract_id': 1459, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10140.69', 'barrier_count': 1, 'bid_price': 0.04750117, 'buy_price': 0.05, 'contract_id': 1459, 'contract_type': 'VANILLALONGCALL', ..."
9,R_75,VANILLALONGPUT,9980.4350,1689911004,0.05,1406m,+517.5950,10498.0300,"[1689911005, 1689995365]",1499,0,0.000093,1689916376,0.036832,0.036832,"{'echo_req': {'contract_id': 1499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10498.0300', 'barrier_count': 1, 'bid_price': 0.03683177, 'buy_price': 0.05, 'contract_id': 1499, 'contract_type': 'VANILLALONGPUT',...","{'echo_req': {'contract_id': 1499, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10498.0300', 'barrier_count': 1, 'bid_price': 0.04968494, 'buy_price': 0.05, 'contract_id': 1499, 'contract_type': 'VANILLALONGPUT',..."
10,R_50,VANILLALONGPUT,10120.6204,1689911284,0.05,19h,-119.8780,10000.7424,"[1689911285, 1689979685]",2079,0,0.001051,1689916378,0.021866,0.021867,"{'echo_req': {'contract_id': 2079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10000.7424', 'barrier_count': 1, 'bid_price': 0.0218663, 'buy_price': 0.05, 'contract_id': 2079, 'contract_type': 'VANILLALONGPUT', ...","{'echo_req': {'contract_id': 2079, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10000.7424', 'barrier_count': 1, 'bid_price': 0.04568998, 'buy_price': 0.05, 'contract_id': 2079, 'contract_type': 'VANILLALONGPUT',..."
14,1HZ10V,VANILLALONGCALL,10040.7900,1689911301,0.05,14h,+21.20,10061.9900,"[1689911301, 16899

In [62]:
# Sold_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

while True:
    
    print(len(Sold_df))
    
    if len(Sold_df) > 20 - 1:
        break
    
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)
    duration_unit = random.choice(['m','h','d'])

    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(max_duration/2, max_duration)

    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Sold_df.loc[len(Sold_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), proposal]
                               

    clear_output(wait=True)
    
Sold_df

20


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json
0,1HZ75V,VANILLALONGPUT,10085.3200,1689916548,0.05,220d,10100.00,10100.0000,"[1689916549, 1708991999]",2599,0,0.000021,"{'echo_req': {'contract_id': 2599, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10100.00', 'barrier_count': 1, 'bid_price': 0.04763472, 'buy_price': 0.05, 'contract_id': 2599, 'contract_type': 'VANILLALONGPUT', '..."
1,R_25,VANILLALONGPUT,10001.8390,1689916552,0.05,184d,8480.000,8480.0000,"[1689916552, 1705881599]",2619,0,0.000300,"{'echo_req': {'contract_id': 2619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '8480.000', 'barrier_count': 1, 'bid_price': 0.04364946, 'buy_price': 0.05, 'contract_id': 2619, 'contract_type': 'VANILLALONGPUT', '..."
2,R_50,VANILLALONGPUT,10225.9867,1689916556,0.05,360d,7800.0000,7800.0000,"[1689916556, 1721087999]",2639,0,0.000059,"{'echo_req': {'contract_id': 2639, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '7800.0000', 'barrier_count': 1, 'bid_price': 0.04560678, 'buy_price': 0.05, 'contract_id': 2639, 'contract_type': 'VANILLALONGPUT', ..."
3,R_100,VANILLALONGCALL,10153.6000,1689916560,0.05,191d,28840.00,28840.0000,"[1689916560, 1706486399]",2659,0,0.000114,"{'echo_req': {'contract_id': 2659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '28840.00', 'barrier_count': 1, 'bid_price': 0.04064937, 'buy_price': 0.05, 'contract_id': 2659, 'contract_type': 'VANILLALONGCALL', ..."
4,R_25,VANILLALONGCALL,10001.8260,1689916564,0.05,292d,9540.000,9540.0000,"[1689916564, 1715212799]",2679,0,0.000044,"{'echo_req': {'contract_id': 2679, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9540.000', 'barrier_count': 1, 'bid_price': 0.04814421, 'buy_price': 0.05, 'contract_id': 2679, 'contract_type': 'VANILLALONGCALL', ..."
5,R_100,VANILLALONGPUT,10158.4500,1689916568,0.05,1395m,-258.70,9899.7500,"[1689916568, 1690000268]",2699,0,0.000469,"{'echo_req': {'contract_id': 2699, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9899.75', 'barrier_count': 1, 'bid_price': 0.04573812, 'buy_price': 0.05, 'contract_id': 2699, 'contract_type': 'VANILLALONGPUT', 'c..."
6,1HZ50V,VANILLALONGPUT,9855.6800,1689916572,0.05,275d,8460.00,8460.0000,"[1689916572, 1713743999]",2719,0,0.000050,"{'echo_req': {'contract_id': 2719, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '8460.00', 'barrier_count': 1, 'bid_price': 0.04639642, 'buy_price': 0.05, 'contract_id': 2719, 'contract_type': 'VANILLALONGPUT', 'c..."
7,1HZ10V,VANILLALONGPUT,10038.5800,1689916576,0.05,13h,+20.40,10058.9800,"[1689916576, 1689963376]",2739,0,0.001781,"{'echo_req': {'contract_id': 2739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10058.98', 'barrier_count': 1, 'bid_price': 0.04880327, 'buy_price': 0.05, 'contract_id': 2739, 'contract_type': 'VANILLALONGPUT', '..."
8,R_75,VANILLALONGCALL,10142.0918,1689916580,0.05,22h,-468.1630,9673.9288,"[1689916580, 1689995780]",2759,0,0.000102,"{'echo_req': {'contract_id': 2759, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9673.9288', 'barrier_count': 1, 'bid_price': 0.04965529, 'buy_price': 0.05, 'contract_id': 2759, 'contract_type': 'VANILLALONGCALL',..."
9,1HZ10V,VANILLALONGPUT,10039.1700,1689916583,0.05,13h,-49.40,9989.7700,"[1689916584, 1689963384]",2799,0,0.

In [69]:
# calc_sold = []
# sold_time = []
# api_sold = []
# sold_prop = []
for i in Sold_df.index:    
    if i >= len(sold_time):       
        clear_output(wait = True)        
        sell_contract(Sold_df.loc[i].ID)        
        proposal = get_proposal(Sold_df.loc[i].ID)
        sold_time.append(proposal['proposal_open_contract']['exit_tick_time'])
        api_sold.append(float(proposal['proposal_open_contract']['sell_price']))
        calc_sold.append( calc_ppp_sold(proposal, 0, 0) )        
        sold_prop.append(proposal)    
Sold_df['Sold_time'] = sold_time
Sold_df['API_sold'] = api_sold
Sold_df['Calc_sold'] = calc_sold
Sold_df['Sold_prop'] = sold_prop
clear_output(wait = True)
print(len(Sold_df), 'Test cases','\nMismatches =',\
      len(Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,8) != 0]),\
      '\nMismatches listed below :')
df = Sold_df[(Sold_df.API_sold) - np.round(Sold_df.Calc_sold,8) != 0]
df

20 Test cases 
Mismatches = 3 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,json,Sold_time,API_sold,Calc_sold,Sold_prop
1,R_25,VANILLALONGPUT,10001.839,1689916552,0.05,184d,8480.000,8480.00,"[1689916552, 1705881599]",2619,0,0.000300,"{'echo_req': {'contract_id': 2619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '8480.000', 'barrier_count': 1, 'bid_price': 0.04364946, 'buy_price': 0.05, 'contract_id': 2619, 'contract_type': 'VANILLALONGPUT', '...",1689916896,0.043719,0.043719,"{'echo_req': {'contract_id': 2619, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'audit_details': {'contract_start': [{'epoch': 1689916548, 'tick': 10001.07, 'tick_display_value': '10001.070'}, {'epoch': 1689916550, 'tick': 1..."
3,R_100,VANILLALONGCALL,10153.600,1689916560,0.05,191d,28840.00,28840.00,"[1689916560, 1706486399]",2659,0,0.000114,"{'echo_req': {'contract_id': 2659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '28840.00', 'barrier_count': 1, 'bid_price': 0.04064937, 'buy_price': 0.05, 'contract_id': 2659, 'contract_type': 'VANILLALONGCALL', ...",1689916902,0.040877,0.040877,"{'echo_req': {'contract_id': 2659, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'audit_details': {'contract_start': [{'epoch': 1689916556, 'tick': 10148.46, 'tick_display_value': '10148.46'}, {'epoch': 1689916558, 'tick': 10..."
7,1HZ10V,VANILLALONGPUT,10038.580,1689916576,0.05,13h,+20.40,10058.98,"[1689916576, 1689963376]",2739,0,0.001781,"{'echo_req': {'contract_id': 2739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10058.98', 'barrier_count': 1, 'bid_price': 0.04880327, 'buy_price': 0.05, 'contract_id': 2739, 'contract_type': 'VANILLALONGPUT', '...",1689917356,0.041502,0.041503,"{'echo_req': {'contract_id': 2739, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'audit_details': {'contract_start': [{'epoch': 1689916574, 'tick': 10038.67, 'tick_display_value': '10038.67'}, {'epoch': 1689916575, 'tick': 10..."


In [68]:
# Expired_df = pd.DataFrame(columns=['Asset', 'Contract_type', 'Spot', 'Spot_time', 'Stake', 'Duration', \
#                                'Displayed_barrier', 'Barrier', 'Start_expiry', 'ID', 'Discount_rate', 'API_PPP', 'API_exp', 'Calc_exp','json'])

contract_types = ["VANILLALONGCALL", "VANILLALONGPUT"]

time_units = ('m')

while True:
    
    print(len(Expired_df))
    
    if len(Expired_df)> 10-1:
        break
            
    contract_type = random.choice(contract_types)

    asset = random.choice(assets.Asset)

    duration_unit = random.choice(time_units)
    duration_unit = 'm'
    if duration_unit == 'm':
        max_duration = 1440

    elif duration_unit == 'h':
        max_duration = 24

    elif duration_unit == 'd':
        max_duration = 365

    i= np.random.randint(6)
        
    duration = np.random.randint(1, 45)
#     duration=1
    contract = get_contract(asset, duration, duration_unit, contract_type)
    
    proposal = contract[0]
    
    discount_rate = contract[1]
    
    displayed_barrier = contract[2]
    
    Expired_df.loc[len(Expired_df)] = [proposal['proposal_open_contract']['underlying'], proposal['proposal_open_contract']['contract_type'], \
                               proposal['proposal_open_contract']['entry_tick'], proposal['proposal_open_contract']['entry_tick_time'], \
                               float(proposal['proposal_open_contract']['buy_price']), str(duration)+duration_unit, displayed_barrier, \
                               float(proposal['proposal_open_contract']['barrier']), [proposal['proposal_open_contract']['date_start'], \
                               proposal['proposal_open_contract']['date_expiry']], proposal['echo_req']['contract_id'], discount_rate, \
                               float(proposal['proposal_open_contract']['number_of_contracts']), 0, 0, proposal]
                               

    clear_output(wait=True)
Expired_df

10


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json
0,1HZ25V,VANILLALONGCALL,10150.2200,1689917259,0.050000,17m,+0.00,10150.2200,"[1689917259, 1689918279]",3899,0,0.008473,0,0,"{'echo_req': {'contract_id': 3899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10150.22', 'barrier_count': 1, 'bid_price': 0.04756098, 'buy_price': 0.05, 'contract_id': 3899, 'contract_type': 'VANILLALONGCALL', ..."
1,1HZ10V,VANILLALONGCALL,10042.5500,1689917263,0.050000,17m,-3.00,10039.5500,"[1689917263, 1689918283]",3919,0,0.012091,0,0,"{'echo_req': {'contract_id': 3919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10039.55', 'barrier_count': 1, 'bid_price': 0.04880035, 'buy_price': 0.05, 'contract_id': 3919, 'contract_type': 'VANILLALONGCALL', ..."
2,1HZ10V,VANILLALONGCALL,10042.8000,1689917267,0.050000,40m,-11.20,10031.6000,"[1689917267, 1689919667]",3939,0,0.004290,0,0,"{'echo_req': {'contract_id': 3939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10031.60', 'barrier_count': 1, 'bid_price': 0.04966945, 'buy_price': 0.05, 'contract_id': 3939, 'contract_type': 'VANILLALONGCALL', ..."
3,R_10,VANILLALONGPUT,10023.1210,1689917270,0.050000,25m,+8.860,10031.9810,"[1689917271, 1689918771]",3979,0,0.005424,0,0,"{'echo_req': {'contract_id': 3979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10031.981', 'barrier_count': 1, 'bid_price': 0.04967076, 'buy_price': 0.05, 'contract_id': 3979, 'contract_type': 'VANILLALONGPUT', ..."
4,1HZ100V,VANILLALONGPUT,9545.1300,1689917274,0.050000,5m,+15.40,9560.5300,"[1689917275, 1689917575]",4019,0,0.002349,0,0,"{'echo_req': {'contract_id': 4019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9560.53', 'barrier_count': 1, 'bid_price': 0.04952559, 'buy_price': 0.05, 'contract_id': 4019, 'contract_type': 'VANILLALONGPUT', 'c..."
5,R_75,VANILLALONGPUT,10175.8896,1689917278,0.050000,19m,+24.2520,10200.1416,"[1689917279, 1689918419]",4059,0,0.001499,0,0,"{'echo_req': {'contract_id': 4059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10200.1416', 'barrier_count': 1, 'bid_price': 0.04880489, 'buy_price': 0.05, 'contract_id': 4059, 'contract_type': 'VANILLALONGPUT',..."
6,1HZ75V,VANILLALONGCALL,10075.2800,1689917283,0.050000,42m,+0.20,10075.4800,"[1689917283, 1689919803]",4119,0,0.001817,0,0,"{'echo_req': {'contract_id': 4119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10075.48', 'barrier_count': 1, 'bid_price': 0.04755214, 'buy_price': 0.05, 'contract_id': 4119, 'contract_type': 'VANILLALONGCALL', ..."
7,1HZ50V,VANILLALONGPUT,9875.7400,1689917289,0.028855,36m,+0.10,9875.8400,"[1689917289, 1689919449]",4159,0,0.001722,0,0,"{'echo_req': {'contract_id': 4159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9875.84', 'barrier_count': 1, 'bid_price': 0.02745165, 'buy_price': 0.028855, 'contract_id': 4159, 'contract_type': 'VANILLALONGPUT'..."
8,1HZ50V,VANILLALONGCALL,9876.2100,1689917293,0.050000,32m,-20.20,9856.0100,"[1689917293, 1689919213]",4179,0,0.001795,0,0,"{'echo_req': {'contract_id': 4179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9856.01', 'barrier_count': 1, 'bid_price': 0.04879936, 'buy_price': 0.05, 'contract_id': 4179, 'contract_type': 'VANILLALONGCALL', '..."
9,R_10,VANILLALONGCALL,10022.0320,16

In [87]:
calc_exp = [] 

exp_time = []

api_exp = []

for i in Expired_df.index:
    
    if i >= len(exp_time):
        
        clear_output(wait = True)
                
        # getting proposals for expired contracts to save API payout
        proposal = get_proposal(Expired_df.loc[i].ID)
        
        if proposal['proposal_open_contract']['is_sold'] == 1:
            
            exp_time.append(proposal['proposal_open_contract']['date_expiry'])

            api_exp.append(float(proposal['proposal_open_contract']['sell_price']))

            # Calculating expired payout to compare againts api
            calc_exp.append( calc_ppp_exp(proposal, 0, 0) )
            
        else:
            
            exp_time.append(0)
            
            api_exp.append(0)

            calc_exp.append(0)
    
Expired_df['Exp_time'] = exp_time

Expired_df['API_exp'] = api_exp

Expired_df['Calc_exp'] = calc_exp

clear_output(wait = True)

print(len(Expired_df), 'Test cases','\nMismatches =',\
      len(Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,8) != 0]),\
      '\nMismatches listed below :')

df = Expired_df[(Expired_df.API_exp) - np.round(Expired_df.Calc_exp,8) != 0]

df

10 Test cases 
Mismatches = 0 
Mismatches listed below :


,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time


In [88]:
Expired_df

,Asset,Contract_type,Spot,Spot_time,Stake,Duration,Displayed_barrier,Barrier,Start_expiry,ID,Discount_rate,API_PPP,API_exp,Calc_exp,json,Exp_time
0,1HZ25V,VANILLALONGCALL,10150.2200,1689917259,0.050000,17m,+0.00,10150.2200,"[1689917259, 1689918279]",3899,0,0.008473,0.000000,0.000000,"{'echo_req': {'contract_id': 3899, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10150.22', 'barrier_count': 1, 'bid_price': 0.04756098, 'buy_price': 0.05, 'contract_id': 3899, 'contract_type': 'VANILLALONGCALL', ...",1689918279
1,1HZ10V,VANILLALONGCALL,10042.5500,1689917263,0.050000,17m,-3.00,10039.5500,"[1689917263, 1689918283]",3919,0,0.012091,0.103862,0.103862,"{'echo_req': {'contract_id': 3919, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10039.55', 'barrier_count': 1, 'bid_price': 0.04880035, 'buy_price': 0.05, 'contract_id': 3919, 'contract_type': 'VANILLALONGCALL', ...",1689918283
2,1HZ10V,VANILLALONGCALL,10042.8000,1689917267,0.050000,40m,-11.20,10031.6000,"[1689917267, 1689919667]",3939,0,0.004290,0.084720,0.084720,"{'echo_req': {'contract_id': 3939, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10031.60', 'barrier_count': 1, 'bid_price': 0.04966945, 'buy_price': 0.05, 'contract_id': 3939, 'contract_type': 'VANILLALONGCALL', ...",1689919667
3,R_10,VANILLALONGPUT,10023.1210,1689917270,0.050000,25m,+8.860,10031.9810,"[1689917271, 1689918771]",3979,0,0.005424,0.000000,0.000000,"{'echo_req': {'contract_id': 3979, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10031.981', 'barrier_count': 1, 'bid_price': 0.04967076, 'buy_price': 0.05, 'contract_id': 3979, 'contract_type': 'VANILLALONGPUT', ...",1689918771
4,1HZ100V,VANILLALONGPUT,9545.1300,1689917274,0.050000,5m,+15.40,9560.5300,"[1689917275, 1689917575]",4019,0,0.002349,0.077414,0.077414,"{'echo_req': {'contract_id': 4019, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9560.53', 'barrier_count': 1, 'bid_price': 0.04952559, 'buy_price': 0.05, 'contract_id': 4019, 'contract_type': 'VANILLALONGPUT', 'c...",1689917575
5,R_75,VANILLALONGPUT,10175.8896,1689917278,0.050000,19m,+24.2520,10200.1416,"[1689917279, 1689918419]",4059,0,0.001499,0.034934,0.034934,"{'echo_req': {'contract_id': 4059, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10200.1416', 'barrier_count': 1, 'bid_price': 0.04880489, 'buy_price': 0.05, 'contract_id': 4059, 'contract_type': 'VANILLALONGPUT',...",1689918419
6,1HZ75V,VANILLALONGCALL,10075.2800,1689917283,0.050000,42m,+0.20,10075.4800,"[1689917283, 1689919803]",4119,0,0.001817,0.177894,0.177894,"{'echo_req': {'contract_id': 4119, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '10075.48', 'barrier_count': 1, 'bid_price': 0.04755214, 'buy_price': 0.05, 'contract_id': 4119, 'contract_type': 'VANILLALONGCALL', ...",1689919803
7,1HZ50V,VANILLALONGPUT,9875.7400,1689917289,0.028855,36m,+0.10,9875.8400,"[1689917289, 1689919449]",4159,0,0.001722,0.000000,0.000000,"{'echo_req': {'contract_id': 4159, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_open_contract': {'account_id': 200221, 'barrier': '9875.84', 'barrier_count': 1, 'bid_price': 0.02745165, 'buy_price': 0.028855, 'contract_id': 4159, 'contract_type': 'VANILLALONGPUT'...",1689919449
8,1HZ50V,VANILLALONGCALL,9876.2100,1689917293,0.050000,32m,-20.20,9856.0100,"[1689917293, 1689919213]",4179,0,0.001795,0.095535,0.095535,"{'echo_req': {'contract_id': 4179, 'proposal_open_contract': 1}, 'msg_type': 'proposal_open_contract', 'proposal_o